# Spacy sentence transformers demo 3



In [2]:
import spacy
import torch
import numpy
from numpy.testing import assert_almost_equal

In [9]:
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("Using gpu!")
    # torch.set_default_tensor_type("torch.cuda.FloatTensor")

Using gpu!


In [10]:
s_model = 'en_core_web_trf'
nlp = spacy.load(s_model)

In [11]:
doc = nlp("Here is some text to encode.")
doc.tensor.shape
doc

Here is some text to encode.

In [12]:
 doc.tensor.shape

(0,)

In [6]:
assert doc.tensor.shape == (7, 768)  # Always has one row per token

AssertionError: 

In [7]:
doc._.trf_word_pieces_  # String values of the wordpieces
doc._.trf_word_pieces  # Wordpiece IDs (note: *not* spaCy's hash values!)
doc._.trf_alignment  # Alignment between spaCy tokens and wordpieces

AttributeError: [E046] Can't retrieve unregistered extension attribute 'trf_word_pieces_'. Did you forget to call the `set_extension` method?

In [ ]:
# The raw transformer output has one row per wordpiece.
assert len(doc._.trf_last_hidden_state) == len(doc._.trf_word_pieces)

In [ ]:
# To avoid losing information, we calculate the doc.tensor attribute such that
# the sum-pooled vectors match (apart from numeric error)
assert_almost_equal(doc.tensor.sum(axis=0), doc._.trf_last_hidden_state.sum(axis=0), decimal=5)
span = doc[2:4]

In [ ]:
# Access the tensor from Span elements (especially helpful for sentences)
assert numpy.array_equal(span.tensor, doc.tensor[2:4])
# .vector and .similarity use the transformer outputs
apple1 = nlp("Apple shares rose on the news.")
apple2 = nlp("Apple sold fewer iPhones this quarter.")
apple3 = nlp("Apple pie is delicious.")
print(apple1[0].similarity(apple2[0]))  # 0.73428553
print(apple1[0].similarity(apple3[0]))  # 0.43365782